# Creating the payoff matrix

## Find unique payoff dates for a sample of bonds
To accurately estimate the term structure of interest rates using coupon bonds, the first step is to identify the unique payment dates for the bonds. These dates will form the columns of the payoff matrix. The rows of the matrix will represent the individual bonds, and the cell values will be the payment amount of the corresponding bond on that specific date.

To demonstrate, we will use an Excel workbook named bond_data_jan21_2025.xlsx from DropBox. The payoff matrix will be constructed using all bonds that mature on the following dates

* February 28$^{th}$ 2025
* July 15$^{th}$ 2025
* August 31$^{st}$ 2025
* January 15$^{th}$ 2026
* February 28$^{th}$ 2026

The settlement date is January 21$^{st}$ 2025.


### Importing libraries, modules, and functions
As in earlier chapters of the volume, modules that are included in the standard Python library are imported. When necessary, other modules or libraries are installed before they are imported.$^{1}$

```
import sys
import requests
from types import ModuleType
from datetime import datetime, date

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
```
---


$^{1}$For more information on <font color='green'>try</font> and <font color='green'>except</font> statements, see “<a href='https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/Control_Statements.html#the-try-and-except'>Control Statements</a>.”

In [1]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module
from datetime import datetime, date

# Import the NumPy library for numerical operations, commonly aliased as np.
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

# Import the pandas library for data manipulation and analysis, aliased as pd.
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

### Adding a custom module and importing functions

#### The custom module <font color='green'>module_basic_concepts_fixed_income</font> custom module  is accessed from Dropbox and named <font color='green'>basic_concepts_fixed_income</font>. Three functions are imported:



*  #### <font color='green'>bond_pay_data()</font> (Chapter Three) Returns NumPy arrays of payment dates and amounts. <a href='https://patrickjhess.github.io/Imported-Functions/bond_pay_data.html#'> View Here</a>

*   #### <font color='green'>check_rank</font> (This Chapter) in the notebook *Using the linalg Module of NumPy* returns the truth value of full column rank of a matrix. <a href='https://patrickjhess.github.io/Imported-Functions/check_rank.html#check-rank-helper-function'>View Here</a>

*   #### <font color='green'>validate_dates</font> (Chapter Two) checks for datetime or date objects and converts datetime objects to date objects. <a href='https://patrickjhess.github.io/Imported-Functions/validate_date.html#validate-date-helper-function'> View Here</a>
```
from basic_concepts_fixed_income import (bond_pay_data,
                                         check_rank,
                                         validate_dates)
```





In [2]:
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (bond_pay_data,
                                         check_rank,
                                         validate_date)

## Getting the sample bonds
<font color='black'> U.S. Treasury notes and bonds data, for settlement on January 21, 2025, was obtained from Fidelity. The Fidelity data, an Excel workbook, is downloaded from DropBox using the Panda's <font color='green'>read_excel()</font> method.$^{2}$

 Three arguments are passed to the method.



1.    The URL address (<font color='green'>url</font>) is required.
2.    Assigning the index column is optional. The maturity dates, located in the first column, are set as the index of the DataFrame  (<font color='green'>index_col='Maturity Date'</font>).
3.    The worksheet name defaults to the first worksheet. The worksheet's name, 'Fidelity Data', is the assigned sheet (<font color='green'>sheet_name='Fidelity Data'</font>)

The <font color='green'>loc</font> is used to access the bonds with five maturity dates.

---

 $^{2}$ <a href='https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/An_Introduction_To_Pandas.html#dataframes-csv-and-excel-files'>Pandas method read_excel</a>

In [3]:
# Define bond maturity dates as a list
maturity_dates = [
    date(2025, 2, 28),
    date(2025, 7, 15),
    date(2025, 8, 31),
    date(2026, 1, 15),
    date(2026, 2, 28),
]

In [4]:
#The full file path.
url='https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/\
bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1'
print(f"Attempting to load data from: {url}")

#Load the data from Excel, using the first column as the index.
try:
    bond_data = pd.read_excel(url, index_col='Maturity Date',sheet_name='Fidelity Data')
    # loc attribute selects bonds with the five maturity dates
    sample_bonds=bond_data.loc[maturity_dates]
    # Display the sample bonds.
    display(sample_bonds)

except FileNotFoundError:
    print("\nERROR: File not found.")
    print("Please check that the 'folder' and 'file' variables are spelled correctly'\
' and that the file exists in that location.")

Attempting to load data from: https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1


,Description,Coupon,Price Bid,Price Ask,Bid Size,Ask Size
Maturity Date,,,,,,
2025-02-28,UNITED STATES TREAS NOTE 1.12500% 02/28/2025,1.125,99.671,99.684,60000.0,60000.0
2025-02-28,UNITED STATES TREAS SER AY-2025 4.6250...,4.625,100.019,100.032,60000.0,60000.0
2025-02-28,UNITED STATES TREAS NOTES 2.7500...,2.750,99.837,99.852,40000.0,40000.0
2025-07-15,UNITED STATES TREAS SER AQ-2025 3.0000...,3.000,99.394,99.406,60000.0,60000.0
2025-08-31,UNITED STATES TREAS SER AC-2025 0.2500...,0.250,97.605,97.610,60000.0,60000.0
2025-08-31,UNITED STATES TREAS SER BG-2025 5.0000...,5.000,100.406,100.426,60000.0,60000.0
2025-08-31,UNITED STATES TREAS SER P-2025 2.7500...,2.750,99.070,99.094,60000.0,60000.0
2026-01-15,UNITED STATES TREAS SER AJ-2026 3.8750...,3.875,99.671,99.688,40000.0,40000.0
2026-02-28,UNITED STATES TREAS SER AY-2026 4.6250...,4.625,100.375,100.387,50000.0,50000.0


### Getting the payments of the eleven bonds

The <font color='green'>bond_pay_data</font> function provides the payment dates and amounts for bonds. This information is stored in the nested list <font color='green'>payment_data</font>.  Each element in this list contains an array of payment dates and amounts.


The last element of <font color='green'>payment_data</font> specifically contains the payment dates and amounts for the 0.5 coupon bond maturing on  February 28$^{th}$ 2026 and its payment dates and amounts are presented.



In [5]:
# The settlement date of the sample_bonds data is 1/21/2025
settlement=date(2025,1,21)

# payment_data created with list comprehension
# Maturity and coupon are from the sample_bonds DataFrame
# Each element of the payment_data is a tuple of a bond's payment dates and amounts
payment_data=[bond_pay_data(maturity,coupon,settlement) for maturity,coupon in
               zip(sample_bonds.index,sample_bonds['Coupon'])]

# Display the last element of payment_data
# The last element of the nested list is returned
# The first and last elements of the payment elements are dates and amounts respectively
print('Payment Dates For 2026-02-28 0.5 bond \n',payment_data[-1][0])
print('Payment Amounts For 2026-02-28 0.5 bond \n',payment_data[-1][1])

Payment Dates For 2026-02-28 0.5 bond 
 [datetime.date(2025, 2, 28) datetime.date(2025, 9, 2)
 datetime.date(2026, 3, 2)]
Payment Amounts For 2026-02-28 0.5 bond 
 [  0.25   0.25 100.25]


The payment amounts and dates for the last bond in the sample provide an opportunity for understanding the <font color='green'>bond_pay_data</font> function. The bond matures 2/28/2026 and pays \$0.50 semi-annually: \\$0.25 every six months. Starting with the maturity date the <font color='green'>bond_pay_data</font> function determines the scheduled pay dates as 2/28/2025, 8/30/21025, and 2/28/2026. Payments, however, can only be made on a business day. If a scheduled date falls on a holiday or weekend, the function assigns the next dusiness day as the payment date. The second and third scheduled dates are adjusted to 9/02/2025 and 3/02/2026, respectively.$^{3}$


---
$^{3}$ View the functions <font color='green'>scheduled_pay_dates</font> <a href='https://patrickjhess.github.io/Imported-Functions/scheduled_pay_dates.html'> (here)</a> and <font color='green'>adjust_bond_pay_dates</font> <a href='(https://patrickjhess.github.io/Imported-Functions/adjust_bond_pay_dates.html)'> (here)</a> developed in Chapter 3 and used by the <font color='green'>bond_pay_data</font> function.

### Determining the unique payment dates of the eleven bonds

### Use set comprehension to create a set of unique payment dates
Set comprehension is utilized to efficiently determine the unique payment dates, which form the columns of the payoff matrix, by automatically eliminating duplicates.$^{4}$

The process involves iterating through <font color='green'>payment_data</font>, a nested list containing the payment dates and amounts for each bond. For every element selected from <font color='green'>payment_data</font>, the unique payment dates of the corresponding bond (the first value of the element) are added to <font color='green'>unique_dates_set</font>.

The bonds' payment schedules are as follows:

* **Three** of the eleven bonds mature on February 28, 2025, with a single payment date: February 28, 2025\.  
* **One** bond matures on July 15, 2025, with a single payment date: July 15, 2025\.  
* **Three** bonds mature on August 31, 2025, with two payment dates: February 28, 2025, and September 2, 2025\.  
* **One** bond matures on January 15, 2026, with two payment dates: July 15, 2025, and January 15, 2026\.  
* **Three** bonds mature on February 28, 2026, with three payment dates: February 28, 2025, September 2, 2025, and March 2, 2026\.

In total, there are five unique payment dates identified.



---
$^{4}$ All elements of a set are unique. [A quick introduction to sets is available here.](https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/Sets.html#sets)


In [6]:
unique_dates_set = {pay_date for element in payment_data for pay_date in element[0]}
unique_dates_set

{datetime.date(2025, 2, 28),
 datetime.date(2025, 7, 15),
 datetime.date(2025, 9, 2),
 datetime.date(2026, 1, 15),
 datetime.date(2026, 3, 2)}

### The function <font color='green'>unique_pay_dates</font>


The <font color='green'>unique_pay_dates</font> function is responsible for creating a dictionary from the payment data generated by <font color='green'>bond_payment_data</font>. This resulting dictionary is essential for constructing the individual payment rows for each bond.

To ensure chronological order, the function first converts the set of unique dates (<font color='green'>unique_dates_set</font>) into a sorted list (<font color='green'>unique_dates_sorted</font>). The final dictionary, named <font color='green'>unique_dates</font>, is then generated from this sorted list.

In the <font color='green'>unique_dates</font> dictionary, the unique dates serve as the keys, and their corresponding values are the index locations of those dates within the sorted list. This structure allows for fast retrieval of column locations using the dictionary's <font color='green'>get</font> method.







In [7]:
def unique_pay_dates(payment_data):
  """
    Creates a dictionary mapping unique dates to a sorted index.
    The unique dates are columns of the payoff matrix

    Args:
        payment_data (list): created with bond_pay_data function

    Returns:
        dict: A dictionary where keys are the unique dates (sorted) and
              values are their corresponding integer index (0, 1, 2...).

    """
  # set comprehension extracts unique dates
  # iterate through every element in payment_data and then through every date in element[0]
  # Sets discard duplicates
  unique_dates_set = {pay_date for element in payment_data for pay_date in element[0]}

  # Convert set to list and sort
  unique_dates_sorted=sorted(list(unique_dates_set))

  # Create dictionary with unique dates as keys and index as values
  # Rows of payments will be derived from the dictionary
  unique_dates={unique_date:index for index,unique_date in enumerate(unique_dates_sorted)}

  return unique_dates

## <font color='green'>Application: Determine unique payment dates for four bonds</font>



<div style="
    border-left: 12px solid green;
    line-height: 1.5;
    padding: 15px">
<br>
Settlement date January 21$^{st}$ 2025 and semi-annual coupons.


|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;January 31$^{st}$ 2025|&nbsp;&nbsp;&nbsp;2.00|
|&nbsp;&nbsp;&nbsp;July 31$^{st}$ 2027|&nbsp;&nbsp;&nbsp;0.00|
|&nbsp;&nbsp;&nbsp;February 28$^{th}$ 2035|&nbsp;&nbsp;&nbsp;4.00|
|&nbsp;&nbsp;&nbsp;August 31$^{st}$ 2025|&nbsp;&nbsp;&nbsp;5.00|





 see [Chapter Four Hints: Determine Unique Payment Dates](https://colab.research.google.com/drive/1hSV2ArS-kEm3soW-sbIBTjWnnD3nMvkh#scrollTo=VELk0_Ht9_y2), and check the [expected results here](https://colab.research.google.com/drive/1hrv5WQPxxuto7jIy1I6_fdyeAyhNCLaY#scrollTo=VELk0_Ht9_y2).

</div>

#### Illustrate creating a row with February 28$^{th}$ 2026 0.5 coupon bond.

For the bond with a 0.5% coupon rate, maturing on February 28$^{th}$ 2026 (the final bond in <font color='green'>sample_bonds</font>), identify the column locations of the bond payment dates.

In [8]:
# unique_pay_dates function is used to return a dictionary of unique dates to column_mapping
column_mapping=unique_pay_dates(payment_data)
print('unique dates = ',column_mapping)
print('last bond dates =',payment_data[-1][0])
# dictionary get method returns the value of the key or none if key is not present
payment_columns=[column_mapping.get(pay_date) for pay_date in payment_data[-1][0]]
print('payment columns =',payment_columns)

unique dates =  {datetime.date(2025, 2, 28): 0, datetime.date(2025, 7, 15): 1, datetime.date(2025, 9, 2): 2, datetime.date(2026, 1, 15): 3, datetime.date(2026, 3, 2): 4}
last bond dates = [datetime.date(2025, 2, 28) datetime.date(2025, 9, 2)
 datetime.date(2026, 3, 2)]
payment columns = [0, 2, 4]


### Create the row for the bond

The payment row's length is determined by the size of the <font color='green'>column_mapping</font>. The row is initially created as a zero-filled NumPy array.

The payment amounts are then assigned to the correct column locations within the row array through iteration.


In [9]:
# Initialize the row array with zero values
row=np.zeros(len(column_mapping))

# Iterate through the payment amounts and the column locations as a tuple
for amount,column in zip(payment_data[-1][1],payment_columns):
  # Check column is not equal to None; i.e. column holds a value
  if not column is None:
    # If column holds a value, assign row column position the correspond payment amount
    row[column]=amount
row

array([  0.25,   0.  ,   0.25,   0.  , 100.25])

## The function <font color='green'>create_payoff_matrix</font>


*   #### The arguments of the function are:
    * #### the DataFrame of bond data
      * #### index is assumed to be the maturity date
      * #### annual coupon/100 assumed to be the column 'Coupon'
    * #### the settlement date assumed to be a date or datetime object.
*   #### Creates the list <font color='green'>payment_rows</font> with payment row (NumPy arrays) as element.
* #### If at least one value of the payment row is non zero, the row is is appended to <font color='green'>payment_rows</font>.
*   #### The array <font color='green'>payoff_matrix</font> is created from the nested list <font color='green'>payment_rows</font>.
*   #### If the payoff matrix is full rank, the payoff matrix and the dates of the columns are returned.



**Purpose:** This function constructs a payoff matrix from bond data, ensuring the matrix is full rank.

**Arguments:**

1. **Bond Data DataFrame:**  
   * The index must represent the maturity date.  
   * The column 'Coupon' is assumed to contain the annual coupon rate, expressed as a decimal (i.e., annual coupon divided by 100).  
2. **Settlement Date:** Must be provided as a date or datetime object.

**Process:**

1. **Payment Row Creation:** It generates a list, <font color='green'>payment_rows</font>, where each element is a payment row (NumPy array).  
2. **Inclusion Criterion:** A payment row is only appended to  <font color='green'>payment_rows</font> if at least one of its values is non-zero.  
3. **Matrix Formation:** The final  <font color='green'>payoff_matrix</font> is constructed from the list <font color='green'>payment_rows</font> holding the payment-row arrays.

**Return Value:**

* The function returns the payment amounts and the corresponding dates for its columns *only if* the generated  <font color='green'>payoff_matrix</font> is full rank.

In [10]:
def create_payoff_matrix(df_bonds,settlement=None,freq=2):
  '''
    Constructs a cash flow matrix (Payoff Matrix) for a portfolio of bonds.

    This function iterates through a DataFrame of bonds, calculates their individual
    cash flows based on maturity and coupon rate, and aligns them into a matrix
    where rows represent bonds and columns represent unique payment dates.

    Parameters
    ----------
    df_bonds : pd.DataFrame
        A DataFrame containing bond data.
        - Index: Maturity dates (datetime-like).
        - Column 'Coupon': Annual coupon rate (e.g., 5.0 for 5%).
    settlement : datetime-like, optional
        The settlement date for calculation. Cash flows before this date are ignored.
        Defaults to date.today() if None.
    freq : int, optional
        Payment frequency per year. Must be 1 (Annual), 2 (Semi-annual),
        4 (Quarterly), or 12 (Monthly). Defaults to 2.

    Returns
    -------
    tuple (np.ndarray, list)
        - np.ndarray: A 2D array of floats (Rows=Bonds, Cols=Dates).
        - list: A sorted list of datetime.date objects corresponding to the matrix columns.

    Raises
    ------
    ValueError
        - If inputs are invalid type.
        - If a generated row contains only zeros (maturity < settlement).
        - If a duplicate bond (row) is detected.
        - If the final matrix is not full rank (linear dependency).


  '''
  import numpy as np
  import pandas as pd
  from datetime import date
  import warnings


  # Validate inputs

  # df_bonds
  if not isinstance((df_bonds),pd.DataFrame):
     raise ValueError("df_bonds must be a Pandas DataFrame")
  df_local=df_bonds.copy()
  # Check if index (maturity dates) are timestamps
  if not isinstance(df_local.index,pd.DatetimeIndex):
    df_local.index=pd.to_datetime(df_local.index)

  #settlement
  if settlement is None:
      settlement = date.today()
  else:
      settlement=validate_date(settlement)

  #freq
  if int(freq) not in [1,2,4,12]:
      print(f"⚠️  your assigned {freq} to freq. It must be (1, 2, 4, or 12)\
      \n      semi-annual assumed (2).")
      freq=int(2)

  # Initialize the list payment_rows
  payment_rows=[]

  # Get payment data for the bonds

  # Only consider bonds with maturities greater than the settlement date
  payment_data=[bond_pay_data(maturity,coupon,settlement,freq=freq) for maturity,coupon in
               zip(df_local.index.date,df_local['Coupon'])if maturity>=settlement]

  # Calculate the unique payment dates for bond with maturities >= settlement
  # create_unique_function takes the list or arrays and returns a chronologically
  # sorted dictionary with keys equal to unique dates and values column location
  column_mapping=unique_pay_dates(payment_data)

  # Iterate through payment_data list
  for data in payment_data:

    # initialize an array of zeros with length equal unique dates length
    this_row=np.zeros(len(column_mapping))

    # Use dictionary get method to determine the column number of the bond's payments
    payment_columns=[column_mapping.get(pay_date) for pay_date in data[0]]
    for columns,payment in zip(payment_columns,data[1]):
     this_row[columns]=payment

    # Check if row is zero
    if np.all(np.isclose(this_row,0)):
      if payment_rows:
        last_valid=payment_rows[-1]
      else:
        last_valid=None
      raise ValueError(f'The row has no payment amounts {new_row}, last valid row {last_valid}')
    payment_rows.append(this_row)

  # Check the rank before
  if not check_rank(np.array(payment_rows)):
    raise ValueError("The payoff matrix is not full rank; check the DataFrame")

  return np.array(payment_rows),list(column_mapping.keys())

## The <font color='green'>create_payoff_matrix</font> function is demonstrated with the eleven bonds of the <font color='green'>sample_bonds</font> DataFrame.

In [11]:
payoffs,column_dates=create_payoff_matrix(sample_bonds,settlement=settlement,freq=2)
print('Rows of the Payoff Matrix\n',
      payoffs,'\n Dates of the columns\n',
      column_dates)

Rows of the Payoff Matrix
 [[100.5625   0.       0.       0.       0.    ]
 [102.3125   0.       0.       0.       0.    ]
 [101.375    0.       0.       0.       0.    ]
 [  0.     101.5      0.       0.       0.    ]
 [  0.125    0.     100.125    0.       0.    ]
 [  2.5      0.     102.5      0.       0.    ]
 [  1.375    0.     101.375    0.       0.    ]
 [  0.       1.9375   0.     101.9375   0.    ]
 [  2.3125   0.       2.3125   0.     102.3125]
 [  1.25     0.       1.25     0.     101.25  ]
 [  0.25     0.       0.25     0.     100.25  ]] 
 Dates of the columns
 [datetime.date(2025, 2, 28), datetime.date(2025, 7, 15), datetime.date(2025, 9, 2), datetime.date(2026, 1, 15), datetime.date(2026, 3, 2)]


## <font color='green'>Application: Determine the payoff matrix for four bonds</font>


<div style="
    border-left: 12px solid green;
    line-height: 1.5;
    padding: 15px">
<br>
Settlement date January 21$^{st}$ 2025 and semi-annual coupons.


|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;January 31$^{st}$ 2025|&nbsp;&nbsp;&nbsp;2.00|
|&nbsp;&nbsp;&nbsp;July 31$^{st}$ 2027|&nbsp;&nbsp;&nbsp;0.00|
|&nbsp;&nbsp;&nbsp;February 28$^{th}$ 2035|&nbsp;&nbsp;&nbsp;4.00|
|&nbsp;&nbsp;&nbsp;August 31$^{st}$ 2027|&nbsp;&nbsp;&nbsp;5.00|





 see [Chapter Four Hints: Determne the payoff matrix for four bonds](https://colab.research.google.com/drive/1hSV2ArS-kEm3soW-sbIBTjWnnD3nMvkh#scrollTo=0l5pIWmmBj2N), and check the [expected results here](https://colab.research.google.com/drive/1hrv5WQPxxuto7jIy1I6_fdyeAyhNCLaY#scrollTo=0l5pIWmmBj2N).

</div>

## <font color='green'>Application: Is the payoff matrix full column rank?</font>


<div style="
    border-left: 12px solid green;
    line-height: 1.5;
    padding: 15px">
<br>
Settlement date January 21$^{st}$ 2025 and semi-annual coupons.


|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;January 31$^{st}$ 2025|&nbsp;&nbsp;&nbsp;2.00|
|&nbsp;&nbsp;&nbsp;July 31$^{st}$ 2027|&nbsp;&nbsp;&nbsp;1.00|
|&nbsp;&nbsp;&nbsp;February 28$^{th}$ 2035|&nbsp;&nbsp;&nbsp;4.00|
|&nbsp;&nbsp;&nbsp;August 31$^{st}$ 2027|&nbsp;&nbsp;&nbsp;5.00|





 see [Chapter Four Hints: Is payoff matrix full rank](https://colab.research.google.com/drive/1hSV2ArS-kEm3soW-sbIBTjWnnD3nMvkh#scrollTo=5p-feMDZB2zT), and check the [expected results here](https://colab.research.google.com/drive/1hrv5WQPxxuto7jIy1I6_fdyeAyhNCLaY#scrollTo=5p-feMDZB2zT).

</div>